<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#IMPORT-ALL-DATASETS" data-toc-modified-id="IMPORT-ALL-DATASETS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>IMPORT ALL DATASETS</a></span></li><li><span><a href="#PERFORM-INITIAL-DATA-CLEANING" data-toc-modified-id="PERFORM-INITIAL-DATA-CLEANING-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>PERFORM INITIAL DATA CLEANING</a></span></li><li><span><a href="#SHOP-LEVEL-FEATURES" data-toc-modified-id="SHOP-LEVEL-FEATURES-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>SHOP-LEVEL FEATURES</a></span></li><li><span><a href="#ITEM-LEVEL-FEATURES" data-toc-modified-id="ITEM-LEVEL-FEATURES-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>ITEM-LEVEL FEATURES</a></span></li><li><span><a href="#DATE-LEVEL-FEATURES" data-toc-modified-id="DATE-LEVEL-FEATURES-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>DATE-LEVEL FEATURES</a></span></li><li><span><a href="#ITEM-DATE-LEVEL-FEATURES" data-toc-modified-id="ITEM-DATE-LEVEL-FEATURES-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>ITEM-DATE-LEVEL FEATURES</a></span></li><li><span><a href="#SHOP-DATE-LEVEL-FEATURES" data-toc-modified-id="SHOP-DATE-LEVEL-FEATURES-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>SHOP-DATE-LEVEL FEATURES</a></span></li><li><span><a href="#SHOP-ITEM-LEVEL-FEATURES" data-toc-modified-id="SHOP-ITEM-LEVEL-FEATURES-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>SHOP-ITEM-LEVEL FEATURES</a></span></li><li><span><a href="#SHOP-ITEM-DATE-LEVEL-FEATURES" data-toc-modified-id="SHOP-ITEM-DATE-LEVEL-FEATURES-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>SHOP-ITEM-DATE-LEVEL FEATURES</a></span></li></ul></div>

In [1]:
import sys
print(sys.version)

3.6.10 |Anaconda, Inc.| (default, Mar 25 2020, 18:53:43) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [364]:
import datetime
import re
import sqlite3
import time

import holidays

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns

from scipy.stats import variation, mode

#set max columns displayed to 100
pd.set_option('display.max_columns',100)

import warnings
warnings.filterwarnings('ignore')

In [56]:
pd.__version__, mpl.__version__, np.__version__

('1.1.3', '3.3.2', '1.15.4')

### IMPORT ALL DATASETS 

In [3]:
sales = pd.read_csv('./Data/competitive-data-science-predict-future-sales/sales_train.csv')

In [57]:
test_df = pd.read_csv('./Data/competitive-data-science-predict-future-sales/test.csv')

In [58]:
items_df = pd.read_csv('./Data/competitive-data-science-predict-future-sales/items.csv')

In [59]:
categories_df = pd.read_csv('./Data/competitive-data-science-predict-future-sales/item_categories.csv')

In [60]:
shops_df = pd.read_csv('./Data/competitive-data-science-predict-future-sales/shops.csv')

In [61]:
macro_df = pd.read_csv('./Data/competitive-data-science-predict-future-sales/macro.csv')

In [62]:
usecols = ['Title','Genre','Developer','Publisher','Release date JP','Release date EU','Release date NA','Addons']
ps4games = pd.read_csv('./Data/competitive-data-science-predict-future-sales/ps4_games.csv', usecols=usecols)

### PERFORM INITIAL DATA CLEANING 

In [4]:
# convert the date column from string to datetime type
sales.date = sales.date.apply(lambda x: datetime.datetime.strptime(x, '%d.%m.%Y'))

In [5]:
# Drop duplicate rows
sales.drop_duplicates(inplace=True)

In [6]:
# Identify duplicate rows by shop-item-date
dupes = sales.loc[sales.duplicated(subset=['shop_id','item_id','date'], keep=False), :]
dupes.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
6959,2013-01-29,0,25,12133,889.0,1.0
6960,2013-01-29,0,25,12133,1389.0,1.0
70686,2013-01-25,0,31,14050,349.0,1.0
70718,2013-01-25,0,31,14050,248.0,1.0
103461,2013-01-27,0,38,15702,549.0,1.0


In [7]:
# Check how often quantity sold was not 1
dupes[dupes.item_cnt_day != 1.0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
103463,2013-01-27,0,38,15702,149.00,-1.0
1154620,2013-12-29,11,27,12133,669.00,2.0
2272100,2014-12-26,23,17,3424,999.00,2.0
2456815,2015-02-17,25,5,21619,332.66,-1.0


In [8]:
# Look into shop-item-date combos with multiple quantities when one quantity was negative
dupes[(dupes.shop_id == 38) & (dupes.item_id == 15702)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
103461,2013-01-27,0,38,15702,549.0,1.0
103463,2013-01-27,0,38,15702,149.0,-1.0


In [9]:
dupes[(dupes.shop_id == 5) & (dupes.item_id == 21619)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2456813,2015-02-17,25,5,21619,499.00,1.0
2456815,2015-02-17,25,5,21619,332.66,-1.0


In [10]:
# Just remove these two pairs of shop-item-dates
dupes = dupes[~((dupes.shop_id == 38) & (dupes.item_id == 15702)) & ~((dupes.shop_id == 5) & (dupes.item_id == 21619))]

In [11]:
# combine remaining shop-item-date-price level values into shop-item-date level values
# by summing the quantity sold and taking the weighted average of price (weighted by quantity)

# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=dupes.loc[x.index, "item_cnt_day"])

dupes = dupes.groupby(['shop_id','item_id','date','date_block_num']).agg({'item_cnt_day':'sum', 'item_price': wm}).reset_index()

In [12]:
dupes.head()

,shop_id,item_id,date,date_block_num,item_cnt_day,item_price
0,12,21619,2015-10-07,33,2.0,449.000000
1,16,12133,2013-03-02,2,2.0,1139.000000
2,16,15702,2013-02-18,1,2.0,349.000000
3,17,3424,2014-12-26,23,3.0,1065.666667
4,25,12133,2013-01-29,0,2.0,1139.000000


In [13]:
# remove the manipulated rows from the original dataframe
sales.drop_duplicates(subset=['shop_id','item_id','date'], keep=False, inplace=True)

# insert the new version of those rows back into the original dataframe
sales = pd.concat([sales, dupes], axis=0, sort=True).reset_index(drop=True)

In [14]:
# remove row with negative price
sales = sales[sales.item_price > 0.]

In [15]:
sales.shape

(2935818, 6)

### SHOP-LEVEL FEATURES 

In [152]:
shops_df.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [154]:
# remove shop_ids 9 and 20 (as they were found to have strange sales trends)
shops_df = shops_df[~(shops_df.shop_id.isin([9,20]))]

In [165]:
# create city of shop column
shops_df.loc[0,'shop_name'] = 'Якутск Орджоникидзе, 56 фран'
shops_df.loc[1,'shop_name'] = 'Якутск ТЦ "Центральный" фран'

In [170]:
shops_df['city'] = shops_df.shop_name.apply(lambda x: x.split()[0])

In [171]:
shops_df.head()

,shop_name,shop_id,city
0,"Якутск Орджоникидзе, 56 фран",0,Якутск
1,"Якутск ТЦ ""Центральный"" фран",1,Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский


In [173]:
shops_df.city.unique()

array(['Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга', 'Коломна',
       'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Ярославль'], dtype=object)

In [174]:
shops_df[shops_df.city == 'Цифровой']

,shop_name,shop_id,city
55,Цифровой склад 1С-Онлайн,55,Цифровой


In [175]:
shops_df.loc[55,'city'] = 'Интернет-магазин'

In [177]:
shops_df.city.unique()

array(['Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга', 'Коломна',
       'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Чехов', 'Ярославль'],
      dtype=object)

In [187]:
# city populations as of 1/1/2020
# (source: https://rosstat.gov.ru/storage/mediabank/CcG8qBhP/mun_obr2020.rar, accessed 11/17/2020):
city_pop = [('РостовНаДону', 1137904.0, '47°14′26″ с. ш. 39°42′38″ в. д.', 'UTC+3'),
            ('Н.Новгород', 1252236.0, '56°19′37″ с. ш. 44°00′27″ в. д.', 'UTC+3'),
            ('Казань', 1257391.0, '55°47′27″ с. ш. 49°06′52″ в. д.', 'UTC+3'),
            ('Новосибирск', 1625631.0, '55°01′ с. ш. 82°55′ в. д.', 'UTC+7'),
            ('Воронеж', 1058261.0, '51°40′18″ с. ш. 39°12′38″ в. д.', 'UTC+3'),
            ('Красноярск', 1093771.0, '56°00′43″ с. ш. 92°52′17″ в. д.', 'UTC+7'),
            ('Ярославль', 608353.0, '57°37′ с. ш. 39°51′ в. д.', 'UTC+3'),
            ('Тюмень', 807271.0, '57°09′ с. ш. 65°32′ в. д.', 'UTC+5'),
            ('Сургут', 380632.0, '61°15′00″ с. ш. 73°26′00″ в. д.', 'UTC+5'),
            ('Омск', 1154507.0, '54°58′ с. ш. 73°23′ в. д.', 'UTC+6'),
            ('Волжский', 323906.0, '48°47′ с. ш. 44°46′ в. д.', 'UTC+4'),
            ('Уфа', 1128787.0, '54°44′ с. ш. 55°58′ в. д.', 'UTC+5'),
            ('Якутск', 322987.0, '62°01′38″ с. ш. 129°43′55″ в. д.', 'UTC+9'),
            ('Балашиха', 507366.0, '55°48′ с. ш. 37°56′ в. д.', 'UTC+3'),
            ('Вологда', 310302.0, '59°13′ с. ш. 39°54′ в. д.', 'UTC+3'),
            ('Жуковский', 107560.0, '55°36′04″ с. ш. 38°06′58″ в. д.', 'UTC+3'),
            ('Калуга', 332039.0, '54°32′00″ с. ш. 36°16′00″ в. д.', 'UTC+3'),
            ('Коломна', 140129.0, '55°05′38″ с. ш. 38°46′05″ в. д.', 'UTC+3'),
            ('Курск', 452976.0, '51°43′ с. ш. 36°11′ в. д.', 'UTC+3'),
            ('Москва', 12678079.0, '55°45′21″ с. ш. 37°37′04″ в. д.', 'UTC+3'),
            ('Мытищи', 235504.0, '55°55′ с. ш. 37°44′ в. д.', 'UTC+3'),
            ('СПб', 5398064.0, '59°57′ с. ш. 30°19′ в. д.', 'UTC+3'),
            ('Самара', 1156659.0, '53°11′ с. ш. 50°07′ в. д.', 'UTC+4'),
            ('Сергиев', 100335.0, '56°18′00″ с. ш. 38°08′00″ в. д.', 'UTC+3'),
            ('Томск', 576624.0, '56°29′19″ с. ш. 84°57′08″ в. д.', 'UTC+7'),
            ('Химки', 259550.0, '55°53′21″ с. ш. 37°26′42″ в. д.', 'UTC+3'),
            ('Чехов', 73321.0, '55°08′42″ с. ш. 37°27′20″ в. д.', 'UTC+3'),
            ('Адыгея', 932629.0, '45°02′00″ с. ш. 38°59′00″ в. д.', 'UTC+3')]

In [236]:
city_df = pd.DataFrame(city_pop, columns=['city','population','geo_coords','time_zone'])
city_df.head()

,city,population,geo_coords,time_zone
0,РостовНаДону,1137904.0,47°14′26″ с. ш. 39°42′38″ в. д.,UTC+3
1,Н.Новгород,1252236.0,56°19′37″ с. ш. 44°00′27″ в. д.,UTC+3
2,Казань,1257391.0,55°47′27″ с. ш. 49°06′52″ в. д.,UTC+3
3,Новосибирск,1625631.0,55°01′ с. ш. 82°55′ в. д.,UTC+7
4,Воронеж,1058261.0,51°40′18″ с. ш. 39°12′38″ в. д.,UTC+3


In [244]:
def lat_lon_to_float(in_coord, degree_sign = u'\N{DEGREE SIGN}', remove = degree_sign + '′' + '″'):
    geo_list = in_coord.translate({ord(char): ' ' for char in remove}) \
                        .replace('с. ш.',',').replace('в. д.','').split(',')
    if len(geo_list[0].split()) == 3:
        geo_lat = float(geo_list[0].split()[0]) + float(geo_list[0].split()[1])/60. + \
                  float(geo_list[0].split()[2])/3600.
    elif len(geo_list[0].split()) == 2:
        geo_lat = float(geo_list[0].split()[0]) + float(geo_list[0].split()[1])/60.
    if len(geo_list[1].split()) == 3:
        geo_lon = float(geo_list[1].split()[0]) + float(geo_list[1].split()[1])/60. + \
                  float(geo_list[1].split()[2])/3600.
    elif len(geo_list[1].split()) == 2:
        geo_lon = float(geo_list[1].split()[0]) + float(geo_list[1].split()[1])/60.
    return geo_lat, geo_lon

In [243]:
all_lat_lons = city_df.geo_coords.apply(lat_lon_to_float)

city_df['geo_lat'] = all_lat_lons.apply(lambda x: x[0])
city_df['geo_lon'] = all_lat_lons.apply(lambda x: x[1])

city_df.head()

,city,population,geo_coords,time_zone,geo_lat,geo_lon
0,РостовНаДону,1137904.0,47°14′26″ с. ш. 39°42′38″ в. д.,UTC+3,47.240556,39.710556
1,Н.Новгород,1252236.0,56°19′37″ с. ш. 44°00′27″ в. д.,UTC+3,56.326944,44.007500
2,Казань,1257391.0,55°47′27″ с. ш. 49°06′52″ в. д.,UTC+3,55.790833,49.114444
3,Новосибирск,1625631.0,55°01′ с. ш. 82°55′ в. д.,UTC+7,55.016667,82.916667
4,Воронеж,1058261.0,51°40′18″ с. ш. 39°12′38″ в. д.,UTC+3,51.671667,39.210556


In [245]:
city_df.drop(columns='geo_coords', inplace=True)

In [248]:
city_df['time_zone'] = city_df.time_zone.apply(lambda x: x[-1]).astype(np.int8)

In [249]:
city_df.head()

,city,population,time_zone,geo_lat,geo_lon
0,РостовНаДону,1137904.0,3,47.240556,39.710556
1,Н.Новгород,1252236.0,3,56.326944,44.007500
2,Казань,1257391.0,3,55.790833,49.114444
3,Новосибирск,1625631.0,7,55.016667,82.916667
4,Воронеж,1058261.0,3,51.671667,39.210556


In [180]:
shops_df[shops_df.city == 'Интернет-магазин']

,shop_name,shop_id,city
12,Интернет-магазин ЧС,12,Интернет-магазин
55,Цифровой склад 1С-Онлайн,55,Интернет-магазин


In [253]:
shops_df = shops_df.merge(city_df, on='city', how='left')
shops_df.head()

,shop_name,shop_id,city,population,time_zone,geo_lat,geo_lon
0,"Якутск Орджоникидзе, 56 фран",0,Якутск,322987.0,9.0,62.027222,129.731944
1,"Якутск ТЦ ""Центральный"" фран",1,Якутск,322987.0,9.0,62.027222,129.731944
2,"Адыгея ТЦ ""Мега""",2,Адыгея,932629.0,3.0,45.033333,38.983333
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,507366.0,3.0,55.800000,37.933333
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,323906.0,4.0,48.783333,44.766667


In [254]:
# create indicator column for online store
shops_df['online_store'] = np.where(shops_df.city == 'Интернет-магазин', 1, 0)

In [266]:
# create column with count of other shops in same city
shops_df['n_other_stores_in_city'] = shops_df.groupby('city').city.transform('count') - 1

In [267]:
shops_df.loc[shops_df[shops_df.city == 'Интернет-магазин'].index,['n_other_stores_in_city']] = np.nan

In [268]:
shops_df.n_other_stores_in_city.value_counts(dropna=False)

1.0     16
0.0     15
11.0    12
2.0      9
3.0      4
NaN      2
Name: n_other_stores_in_city, dtype: int64

In [289]:
# Add column for total transaction counts (at item-date level) by shop
shop_total_tx_cts = sales.groupby('shop_id').size().reset_index().rename(columns={0:'shop_total_tx_cnt'})
shops_df = shops_df.merge(shop_total_tx_cts, on='shop_id', how='left')

In [290]:
shops_df.head()

,shop_name,shop_id,city,population,time_zone,geo_lat,geo_lon,online_store,n_other_stores_in_city,shop_total_tx_cnt
0,"Якутск Орджоникидзе, 56 фран",0,Якутск,322987.0,9.0,62.027222,129.731944,0,3.0,9857
1,"Якутск ТЦ ""Центральный"" фран",1,Якутск,322987.0,9.0,62.027222,129.731944,0,3.0,5678
2,"Адыгея ТЦ ""Мега""",2,Адыгея,932629.0,3.0,45.033333,38.983333,0,0.0,25991
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,507366.0,3.0,55.800000,37.933333,0,0.0,25532
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,323906.0,4.0,48.783333,44.766667,0,0.0,38242


In [294]:
# Add column for number of unique items sold by shop
unique_items_by_shop = sales.groupby('shop_id')['item_id'].unique()
unique_item_cts_by_shop = unique_items_by_shop.map(len).reset_index().rename(columns={'item_id':'shop_unique_item_cts'})
shops_df = shops_df.merge(unique_item_cts_by_shop, on='shop_id', how='left')
shops_df.head()

,shop_name,shop_id,city,population,time_zone,geo_lat,geo_lon,online_store,n_other_stores_in_city,shop_total_tx_cnt,shop_unique_item_cts
0,"Якутск Орджоникидзе, 56 фран",0,Якутск,322987.0,9.0,62.027222,129.731944,0,3.0,9857,3600
1,"Якутск ТЦ ""Центральный"" фран",1,Якутск,322987.0,9.0,62.027222,129.731944,0,3.0,5678,2523
2,"Адыгея ТЦ ""Мега""",2,Адыгея,932629.0,3.0,45.033333,38.983333,0,0.0,25991,4849
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,507366.0,3.0,55.800000,37.933333,0,0.0,25532,5261
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,323906.0,4.0,48.783333,44.766667,0,0.0,38242,7309


In [299]:
# Add column for number of unique categories of items sold by shop
sales_w_item_cat_id = sales[['item_id','shop_id',]].merge(items_df, on="item_id", how='left')
unique_cat_cts_by_shop = (sales_w_item_cat_id.groupby('shop_id')['item_category_id'].nunique() \
                          .reset_index().rename(columns={'item_category_id':"shop_unique_cat_cts"}))
shops_df = shops_df.merge(unique_cat_cts_by_shop, on='shop_id', how='left')

In [300]:
shops_df.head()

,shop_name,shop_id,city,population,time_zone,geo_lat,geo_lon,online_store,n_other_stores_in_city,shop_total_tx_cnt,shop_unique_item_cts,shop_unique_cat_cts
0,"Якутск Орджоникидзе, 56 фран",0,Якутск,322987.0,9.0,62.027222,129.731944,0,3.0,9857,3600,48
1,"Якутск ТЦ ""Центральный"" фран",1,Якутск,322987.0,9.0,62.027222,129.731944,0,3.0,5678,2523,41
2,"Адыгея ТЦ ""Мега""",2,Адыгея,932629.0,3.0,45.033333,38.983333,0,0.0,25991,4849,59
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,507366.0,3.0,55.800000,37.933333,0,0.0,25532,5261,58
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,323906.0,4.0,48.783333,44.766667,0,0.0,38242,7309,61


### ITEM-LEVEL FEATURES

In [351]:
# Calculate the coefficient of variation of price for each item separately
item_level_features = (sales.groupby('item_id')['item_price'].agg(variation)
                       .reset_index().rename(columns={'item_price':'coef_var_price'}))

In [352]:
# Calculate the mean absolute deviation of quantity sold for each item
item_level_features['quant_mean_abs_dev'] = sales.groupby('item_id')['item_cnt_day'].mad().values

In [353]:
# Add item_name, item_category_id, item_category_name columns
item_level_features = item_level_features.merge(items_df, on='item_id', how='left')
item_level_features = item_level_features.merge(categories_df, on='item_category_id', how='left')
item_level_features.head()

,item_id,coef_var_price,quant_mean_abs_dev,item_name,item_category_id,item_category_name
0,0,0.000000,0.0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40,Кино - DVD
1,1,0.000000,0.0,!ABBYY FineReader 12 Professional Edition Full...,76,Программы - Для дома и офиса (Цифра)
2,2,0.000000,0.0,***В ЛУЧАХ СЛАВЫ (UNV) D,40,Кино - DVD
3,3,0.265823,0.0,***ГОЛУБАЯ ВОЛНА (Univ) D,40,Кино - DVD
4,4,0.000000,0.0,***КОРОБКА (СТЕКЛО) D,40,Кино - DVD


In [354]:
# Create broad category name column
item_level_features['item_category_broad'] = item_level_features.item_category_name.apply(lambda x: x.split()[0])

In [355]:
# Create modified category name column where items related to same kind of game console are grouped
def group_game_consoles(cat_name):
    if 'PS2' in cat_name:
        return 'PS2'
    elif 'PS3' in cat_name:
        return 'PS3'
    elif 'PS4' in cat_name:
        return 'PS4'
    elif 'PSP' in cat_name:
        return 'PSP'
    elif 'PSVita' in cat_name:
        return 'PSVita'
    elif 'XBOX 360' in cat_name:
        return 'XBOX 360'
    elif 'XBOX ONE' in cat_name:
        return 'XBOX ONE'
    elif 'Игры PC' in cat_name:
        return 'Игры PC'
    else:
        return cat_name

item_level_features['item_cat_grouped_by_game_console'] = (item_level_features.item_category_name \
                                                           .apply(group_game_consoles))

In [356]:
# Create indicator column for whether item is digital
item_level_features['digital_item'] = np.where((item_level_features.item_category_name.str.contains("Цифра")) | \
                                              (item_level_features.item_category_name.str.contains("MP3")), 1, 0)

In [357]:
# create column for number of shops that sold the item
item_n_shops_selling = (sales.groupby('item_id').shop_id.nunique() \
                        .reset_index().rename(columns={'shop_id':"item_unique_shop_cts"}))
item_level_features = item_level_features.merge(item_n_shops_selling, on='item_id', how='left')
item_level_features.head()

,item_id,coef_var_price,quant_mean_abs_dev,item_name,item_category_id,item_category_name,item_category_broad,item_cat_grouped_by_game_console,digital_item,item_unique_shop_cts
0,0,0.000000,0.0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40,Кино - DVD,Кино,Кино - DVD,0,1
1,1,0.000000,0.0,!ABBYY FineReader 12 Professional Edition Full...,76,Программы - Для дома и офиса (Цифра),Программы,Программы - Для дома и офиса (Цифра),1,1
2,2,0.000000,0.0,***В ЛУЧАХ СЛАВЫ (UNV) D,40,Кино - DVD,Кино,Кино - DVD,0,1
3,3,0.265823,0.0,***ГОЛУБАЯ ВОЛНА (Univ) D,40,Кино - DVD,Кино,Кино - DVD,0,1
4,4,0.000000,0.0,***КОРОБКА (СТЕКЛО) D,40,Кино - DVD,Кино,Кино - DVD,0,1


In [358]:
# create column for number of unique days on which item was sold
item_n_days_sold = (sales.groupby('item_id')['date'].nunique() \
                    .reset_index().rename(columns={'date':"item_unique_date_cts"}))
item_level_features = item_level_features.merge(item_n_days_sold, on='item_id', how='left')
item_level_features.head()

,item_id,coef_var_price,quant_mean_abs_dev,item_name,item_category_id,item_category_name,item_category_broad,item_cat_grouped_by_game_console,digital_item,item_unique_shop_cts,item_unique_date_cts
0,0,0.000000,0.0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40,Кино - DVD,Кино,Кино - DVD,0,1,1
1,1,0.000000,0.0,!ABBYY FineReader 12 Professional Edition Full...,76,Программы - Для дома и офиса (Цифра),Программы,Программы - Для дома и офиса (Цифра),1,1,6
2,2,0.000000,0.0,***В ЛУЧАХ СЛАВЫ (UNV) D,40,Кино - DVD,Кино,Кино - DVD,0,1,2
3,3,0.265823,0.0,***ГОЛУБАЯ ВОЛНА (Univ) D,40,Кино - DVD,Кино,Кино - DVD,0,1,2
4,4,0.000000,0.0,***КОРОБКА (СТЕКЛО) D,40,Кино - DVD,Кино,Кино - DVD,0,1,1


In [362]:
item_dates = (sales[['item_id','date']].drop_duplicates(subset=['item_id','date']) \
              .sort_values(by=['item_id','date']).reset_index(drop=True))
item_dates.head()

,item_id,date
0,0,2014-09-01
1,1,2014-04-02
2,1,2014-04-04
3,1,2014-07-06
4,1,2014-08-04


In [363]:
# create column for time elapsed since previous sale of same item
item_dates['days_since_prev_sale'] = item_dates.groupby('item_id').date.diff().dt.days
item_dates.head()

,item_id,date,days_since_prev_sale
0,0,2014-09-01,NaN
1,1,2014-04-02,NaN
2,1,2014-04-04,2.0
3,1,2014-07-06,93.0
4,1,2014-08-04,29.0


In [370]:
item_gap_bw_days_stats = item_dates.groupby('item_id').agg(
    item_gap_days_mean=('days_since_prev_sale',np.mean),
    item_gap_days_std=('days_since_prev_sale',np.std),
    item_gap_days_median=('days_since_prev_sale',np.median),
    item_gap_days_min=('days_since_prev_sale',np.min),
    item_gap_days_max=('days_since_prev_sale',np.max)).reset_index()

item_level_features = item_level_features.merge(item_gap_bw_days_stats, on='item_id', how='left')
item_level_features.head()

,item_id,coef_var_price,quant_mean_abs_dev,item_name,item_category_id,item_category_name,item_category_broad,item_cat_grouped_by_game_console,digital_item,item_unique_shop_cts,item_unique_date_cts,item_gap_days_mean,item_gap_days_std,item_gap_days_median,item_gap_days_min,item_gap_days_max
0,0,0.000000,0.0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40,Кино - DVD,Кино,Кино - DVD,0,1,1,NaN,NaN,NaN,NaN,NaN
1,1,0.000000,0.0,!ABBYY FineReader 12 Professional Edition Full...,76,Программы - Для дома и офиса (Цифра),Программы,Программы - Для дома и офиса (Цифра),1,1,6,40.2,33.086251,38.0,2.0,93.0
2,2,0.000000,0.0,***В ЛУЧАХ СЛАВЫ (UNV) D,40,Кино - DVD,Кино,Кино - DVD,0,1,2,80.0,NaN,80.0,80.0,80.0
3,3,0.265823,0.0,***ГОЛУБАЯ ВОЛНА (Univ) D,40,Кино - DVD,Кино,Кино - DVD,0,1,2,52.0,NaN,52.0,52.0,52.0
4,4,0.000000,0.0,***КОРОБКА (СТЕКЛО) D,40,Кино - DVD,Кино,Кино - DVD,0,1,1,NaN,NaN,NaN,NaN,NaN


### DATE-LEVEL FEATURES

In [18]:
# create column with dates from start to end of training period
date_level_features = (sales[['date','date_block_num']].drop_duplicates(subset=['date','date_block_num']) \
                       .sort_values(by='date').reset_index(drop=True))

In [19]:
# create year column
date_level_features['year'] = date_level_features.date.dt.year.astype('category')

In [20]:
# create month column
date_level_features['month'] = date_level_features.date_block_num % 12

In [21]:
# create days in month column
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
date_level_features['days_in_mon'] = date_level_features['month'].map(days).astype(np.int8)

In [22]:
# create sine and cosine of month columns
date_level_features['month_sin'] = np.sin((date_level_features.month)*(2.*np.pi/12))
date_level_features['month_cos'] = np.cos((date_level_features.month)*(2.*np.pi/12))

In [23]:
# create day of week column
date_level_features['day_of_week'] = date_level_features.date.dt.weekday

In [24]:
# create sine and cosine of day of week columns
date_level_features['dow_sin'] = np.sin((date_level_features.day_of_week)*(2.*np.pi/7))
date_level_features['dow_cos'] = np.cos((date_level_features.day_of_week)*(2.*np.pi/7))

In [25]:
# create indicator column for weekend days
date_level_features['is_weekend'] = np.where(date_level_features.day_of_week < 5., 0, 1)

In [26]:
# create quarter of year column
date_level_features['quarter_of_year'] = date_level_features.date.dt.quarter

In [27]:
# create sine and cosine of quarter of year columns
date_level_features['qoy_sin'] = np.sin((date_level_features.quarter_of_year)*(2.*np.pi/4))
date_level_features['qoy_cos'] = np.cos((date_level_features.quarter_of_year)*(2.*np.pi/4))

In [143]:
# create continuous quarter of year column
date_level_features['quarter_counter'] = (date_level_features.date_block_num // 3) + 1

In [31]:
# create week of year column
date_level_features['week_of_year'] = date_level_features.date.dt.isocalendar().week

In [32]:
# create sine and cosine of week of year columns
date_level_features['woy_sin'] = np.sin((date_level_features.week_of_year)*(2.*np.pi/52))
date_level_features['woy_cos'] = np.cos((date_level_features.week_of_year)*(2.*np.pi/52))

In [35]:
# Russian public holidays in 2013, 2014 and 2015
for date, name in sorted(holidays.Russia(years=[2013,2014,2015]).items()):
    print(date, name)

2013-01-01 Новый год
2013-01-02 Новый год
2013-01-03 Новый год
2013-01-04 Новый год
2013-01-05 Новый год
2013-01-06 Новый год
2013-01-07 Православное Рождество
2013-01-08 Новый год
2013-02-23 День защитника отечества
2013-03-08 День женщин
2013-05-01 Праздник Весны и Труда
2013-05-09 День Победы
2013-06-12 День России
2013-11-04 День народного единства
2014-01-01 Новый год
2014-01-02 Новый год
2014-01-03 Новый год
2014-01-04 Новый год
2014-01-05 Новый год
2014-01-06 Новый год
2014-01-07 Православное Рождество
2014-01-08 Новый год
2014-02-23 День защитника отечества
2014-03-08 День женщин
2014-05-01 Праздник Весны и Труда
2014-05-09 День Победы
2014-06-12 День России
2014-11-04 День народного единства
2015-01-01 Новый год
2015-01-02 Новый год
2015-01-03 Новый год
2015-01-04 Новый год
2015-01-05 Новый год
2015-01-06 Новый год
2015-01-07 Православное Рождество
2015-01-08 Новый год
2015-02-23 День защитника отечества
2015-03-08 День женщин
2015-05-01 Праздник Весны и Труда
2015-05-09 День 

In [39]:
public_holidays = holidays.Russia(years=[2013,2014,2015])

In [42]:
# create indicator column for whether date is a public holiday
date_level_features['holiday'] = date_level_features.date.isin(public_holidays).astype(np.int8)

In [43]:
date_level_features.holiday.value_counts()

0    993
1     41
Name: holiday, dtype: int64

In [48]:
# create indicator column for major events
olympics = pd.date_range(start='2/7/2014', end='2/23/2014').to_series().values
world_cup = pd.date_range(start='6/12/2014', end='7/13/2014').to_series().values
major_events = np.concatenate([olympics, world_cup])

In [52]:
date_level_features['major_event'] = date_level_features.date.isin(major_events).astype(np.int8)
date_level_features.major_event.value_counts()

0    985
1     49
Name: major_event, dtype: int64

In [67]:
# convert the date column in macro_df from string to datetime type
macro_df['date'] = pd.to_datetime(macro_df.timestamp)

In [70]:
# subset macro_df dataset to relevant period
macro_df_2013_2015 = macro_df[(macro_df.date >= datetime.datetime(2013, 1, 1)) 
                              & (macro_df.date <= datetime.datetime(2015, 11, 30))]

In [136]:
# identify columns in macro_df dataset that have no null values
macro_nulls = macro_df_2013_2015.isnull().sum(axis=0).reset_index().rename(columns={'index':'column', 0:'count'})
cols_wo_nulls = np.array(macro_nulls[macro_nulls['count'] == 0]['column'])
cols_wo_nulls

array(['timestamp', 'oil_urals', 'gdp_quart', 'gdp_quart_growth', 'cpi',
       'ppi', 'gdp_deflator', 'balance_trade', 'balance_trade_growth',
       'usdrub', 'eurrub', 'brent', 'net_capital_export', 'gdp_annual',
       'gdp_annual_growth', 'average_provision_of_build_contract',
       'average_provision_of_build_contract_moscow', 'rts', 'micex',
       'micex_rgbi_tr', 'micex_cbi_tr', 'deposits_value',
       'deposits_growth', 'deposits_rate', 'mortgage_value',
       'mortgage_growth', 'mortgage_rate', 'income_per_cap', 'salary',
       'salary_growth', 'fixed_basket', 'retail_trade_turnover',
       'retail_trade_turnover_per_cap', 'retail_trade_turnover_growth',
       'labor_force', 'unemployment', 'employment',
       'invest_fixed_capital_per_cap', 'invest_fixed_assets',
       'pop_natural_increase', 'childbirth', 'mortality',
       'average_life_exp', 'rent_price_4+room_bus',
       'rent_price_3room_bus', 'rent_price_2room_bus',
       'rent_price_1room_bus', 'rent_price

Frequency of update of each of these columns:  

**daily**: brent, usdrub, eurrub, rts, micex, micex_cbi_tr, micex_rgbi_tr  
**monthly**: oil_urals, cpi, ppi, balance_trade, balance_trade_growth (only 12 unique), deposits_value, deposits_growth, deposits_rate, mortgage_value, mortgage_growth, mortgage_rate, income_per_cap, fixed_basket, rent_price_4+room_bus, rent_price_3room_bus, rent_price_2room_bus, rent_price_1room_bus, rent_price_3room_eco, rent_price_2room_eco, rent_price_1room_eco  
**quarterly**: average_provision_of_build_contract, average_provision_of_build_contract_moscow, gdp_quart, gdp_quart_growth  
**annual**: gdp_deflator, gdp_annual, gdp_annual_growth, salary, salary_growth, retail_trade_turnover, retail_trade_turnover_growth, retail_trade_turnover_per_cap, labor_force, unemployment, employment, invest_fixed_capital_per_cap, invest_fixed_assets, pop_natural_increase, childbirth, mortality, average_life_exp, load_of_teachers_school_per_teacher, students_state_oneshift, modern_education_share, old_education_build_share, provision_nurse, load_on_doctors, turnover_catering_per_cap, seats_theather_rfmin_per_100000_cap, bandwidth_sports, apartment_fund_sqm

In [138]:
# add these columns to the date_level_features dataframe
cols_wo_nulls = cols_wo_nulls[cols_wo_nulls != 'timestamp']
date_level_features = date_level_features.merge(macro_df_2013_2015[cols_wo_nulls], on='date', how='left')

### ITEM-DATE-LEVEL FEATURES

In [301]:
# create column with quantity sold by item-date
item_date_level_features = (sales.groupby(['item_id','date'])['item_cnt_day'].sum().reset_index() \
                            .rename(columns={'item_cnt_day':'item_qty_sold_day'}))

In [302]:
item_date_level_features.head()

,item_id,date,item_qty_sold_day
0,0,2014-09-01,1.0
1,1,2014-04-02,1.0
2,1,2014-04-04,1.0
3,1,2014-07-06,1.0
4,1,2014-08-04,1.0


In [316]:
# create lag column for quantity sold, grouped by item
shifted = item_date_level_features.groupby('item_id').item_qty_sold_day.shift()
item_date_level_features = item_date_level_features.join(shifted.rename('item_qty_sold_day_lag'))

In [320]:
item_date_level_features.head()

,item_id,date,item_qty_sold_day,item_qty_sold_day_lag
0,0,2014-09-01,1.0,NaN
1,1,2014-04-02,1.0,NaN
2,1,2014-04-04,1.0,1.0
3,1,2014-07-06,1.0,1.0
4,1,2014-08-04,1.0,1.0


In [325]:
# create column for time elapsed since previous sale of same item
item_date_level_features['item_days_since_prev_sale'] = (item_date_level_features.groupby('item_id') \
                                                         .date.diff().dt.days)
item_date_level_features.head()

,item_id,date,item_qty_sold_day,item_qty_sold_day_lag,item_days_since_prev_sale
0,0,2014-09-01,1.0,NaN,NaN
1,1,2014-04-02,1.0,NaN,NaN
2,1,2014-04-04,1.0,1.0,2.0
3,1,2014-07-06,1.0,1.0,93.0
4,1,2014-08-04,1.0,1.0,29.0


In [329]:
# create column for time elapsed since first sale date of same item
item_date_level_features['item_days_since_first_sale'] = ((item_date_level_features['date'] \
                                                           - item_date_level_features.groupby('item_id')['date'] \
                                                           .transform('first')).dt.days)
item_date_level_features.head()

,item_id,date,item_qty_sold_day,item_qty_sold_day_lag,item_days_since_prev_sale,item_days_since_first_sale
0,0,2014-09-01,1.0,NaN,NaN,0
1,1,2014-04-02,1.0,NaN,NaN,0
2,1,2014-04-04,1.0,1.0,2.0,2
3,1,2014-07-06,1.0,1.0,93.0,95
4,1,2014-08-04,1.0,1.0,29.0,124


In [331]:
# create indicator column for first week of sale of item
item_date_level_features['item_first_week'] = ((item_date_level_features['item_days_since_first_sale'] <= 6) \
                                               .astype(np.int8))
# create indicator column for first month of sale of item
item_date_level_features['item_first_month'] = ((item_date_level_features['item_days_since_first_sale'] <= 30) \
                                               .astype(np.int8))
item_date_level_features.head()

,item_id,date,item_qty_sold_day,item_qty_sold_day_lag,item_days_since_prev_sale,item_days_since_first_sale,item_first_week,item_first_month
0,0,2014-09-01,1.0,NaN,NaN,0,1,1
1,1,2014-04-02,1.0,NaN,NaN,0,1,1
2,1,2014-04-04,1.0,1.0,2.0,2,1,1
3,1,2014-07-06,1.0,1.0,93.0,95,0,0
4,1,2014-08-04,1.0,1.0,29.0,124,0,0


### SHOP-DATE-LEVEL FEATURES

In [332]:
# create column with quantity sold by shop-date
shop_date_level_features = (sales.groupby(['shop_id','date'])['item_cnt_day'].sum().reset_index() \
                            .rename(columns={'item_cnt_day':'shop_qty_sold_day'}))

In [333]:
shop_date_level_features.head()

,shop_id,date,shop_qty_sold_day
0,0,2013-01-03,203.0
1,0,2013-01-04,352.0
2,0,2013-01-05,292.0
3,0,2013-01-06,209.0
4,0,2013-01-07,132.0


In [335]:
# create lag column for quantity sold, grouped by shop
shifted = shop_date_level_features.groupby('shop_id').shop_qty_sold_day.shift()
shop_date_level_features = shop_date_level_features.join(shifted.rename('shop_qty_sold_day_lag'))
shop_date_level_features.head()

,shop_id,date,shop_qty_sold_day,shop_qty_sold_day_lag
0,0,2013-01-03,203.0,NaN
1,0,2013-01-04,352.0,203.0
2,0,2013-01-05,292.0,352.0
3,0,2013-01-06,209.0,292.0
4,0,2013-01-07,132.0,209.0


In [336]:
# create column for time elapsed since first sale date in same shop (i.e., age of shop)
shop_date_level_features['shop_days_since_first_sale'] = ((shop_date_level_features['date'] \
                                                           - shop_date_level_features.groupby('shop_id')['date'] \
                                                           .transform('first')).dt.days)
shop_date_level_features.head()

,shop_id,date,shop_qty_sold_day,shop_qty_sold_day_lag,shop_days_since_first_sale
0,0,2013-01-03,203.0,NaN,0
1,0,2013-01-04,352.0,203.0,1
2,0,2013-01-05,292.0,352.0,2
3,0,2013-01-06,209.0,292.0,3
4,0,2013-01-07,132.0,209.0,4


In [337]:
# create indicator column for first week of sale at shop
shop_date_level_features['shop_first_week'] = ((shop_date_level_features['shop_days_since_first_sale'] <= 6) \
                                               .astype(np.int8))
# create indicator column for first month of sale at shop
shop_date_level_features['shop_first_month'] = ((shop_date_level_features['shop_days_since_first_sale'] <= 30) \
                                               .astype(np.int8))
shop_date_level_features.head()

,shop_id,date,shop_qty_sold_day,shop_qty_sold_day_lag,shop_days_since_first_sale,shop_first_week,shop_first_month
0,0,2013-01-03,203.0,NaN,0,1,1
1,0,2013-01-04,352.0,203.0,1,1,1
2,0,2013-01-05,292.0,352.0,2,1,1
3,0,2013-01-06,209.0,292.0,3,1,1
4,0,2013-01-07,132.0,209.0,4,1,1


### SHOP-ITEM-LEVEL FEATURES 

In [380]:
# create column with quantity sold by shop-item
shop_item_level_features = (sales.groupby(['shop_id','item_id'])['item_cnt_day'].sum().reset_index() \
                            .rename(columns={'item_cnt_day':'shop_item_qty_sold'}))
shop_item_level_features.head()

,shop_id,item_id,shop_item_qty_sold
0,0,30,31.0
1,0,31,11.0
2,0,32,16.0
3,0,33,6.0
4,0,35,15.0


In [381]:
# create column with rankings of shops by quantity sold for each item
ranks = shop_item_level_features.groupby('item_id')['shop_item_qty_sold'].rank('min', ascending=False)
shop_item_level_features = shop_item_level_features.join(ranks.rename("shop_item_qty_rank"), how='left')
shop_item_level_features.head()

,shop_id,item_id,shop_item_qty_sold,shop_item_qty_rank
0,0,30,31.0,22.0
1,0,31,11.0,37.0
2,0,32,16.0,34.0
3,0,33,6.0,40.0
4,0,35,15.0,4.0


### SHOP-ITEM-DATE-LEVEL FEATURES